In [5]:
#Import necessary libraries
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from transformers import BertTokenizer, TFBertModel
data = pd.read_csv('bot_detection_data.csv')  # Make sure to update this path

# Numeric features and standardization
numeric_features = data[['Retweet Count', 'Mention Count', 'Follower Count']]
scaler = StandardScaler()
numeric_features = scaler.fit_transform(numeric_features)

# Convert 'Verified' to integer
verified = data['Verified'].astype(int).values.reshape(-1, 1)

# Combine numeric features and 'Verified'
additional_features = np.concatenate((numeric_features, verified), axis=1)

# Split data
train_texts, test_texts, train_labels, test_labels, train_features, test_features = train_test_split(
    data['Tweet'], data['Bot Label'], additional_features, test_size=0.2, random_state=42
)

#Tokenization and Encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def encode_texts(texts):
    return tokenizer(texts, padding=True, truncation=True, max_length=128, return_tensors="tf")

train_encodings = encode_texts(train_texts.tolist())
test_encodings = encode_texts(test_texts.tolist())

#Define the custom model
class BertWithFeatures(tf.keras.Model):
    def __init__(self, bert_model, num_additional_features):
        super(BertWithFeatures, self).__init__()
        self.bert = bert_model
        self.dense = tf.keras.layers.Dense(32, activation='relu')
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.classifier = tf.keras.layers.Dense(1, activation='sigmoid')
        self.additional_features_layer = tf.keras.layers.Dense(num_additional_features, activation='relu')

    def call(self, inputs):
        input_ids, attention_mask, additional_features = inputs
        
        # BERT model
        bert_output = self.bert(input_ids, attention_mask=attention_mask)[1]
        bert_output = self.dropout(bert_output)
        
        # Additional features
        additional_features = self.additional_features_layer(additional_features)
        
        # Concatenate BERT output with additional features
        combined = tf.concat([bert_output, additional_features], axis=-1)
        combined = self.dense(combined)
        
        # Classifier
        return self.classifier(combined)

# Initialize the model
bert_base = TFBertModel.from_pretrained('bert-base-uncased')
model = BertWithFeatures(bert_base, additional_features.shape[1])

#Compile and Train the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Prepare inputs
train_input = {
    'input_ids': train_encodings['input_ids'], 
    'attention_mask': train_encodings['attention_mask'], 
    'additional_features': train_features
}
test_input = {
    'input_ids': test_encodings['input_ids'], 
    'attention_mask': test_encodings['attention_mask'], 
    'additional_features': test_features
}

# Train the model
model.fit(
    (train_input['input_ids'], train_input['attention_mask'], train_input['additional_features']), 
    train_labels.values, 
    validation_split=0.1,
    batch_size=32, 
    epochs=3
)

#Evaluate the model
model.evaluate(
    (test_input['input_ids'], test_input['attention_mask'], test_input['additional_features']),
    test_labels.values
)


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Epoch 1/3
1125/1125 [==============================] - 2757s 2s/step - loss: 0.6958 - accuracy: 0.5018 - val_loss: 0.6931 - val_accuracy: 0.5105
Epoch 2/3
1125/1125 [==============================] - 2747s 2s/step - loss: 0.6934 - accuracy: 0.5008 - val_loss: 0.6931 - val_accuracy: 0.5105
Epoch 3/3
313/313 [==============================] - 184s 579ms/step - loss: 0.6931 - accuracy: 0.5037


[0.6931256055831909, 0.5037000179290771]